# 2. Signatures and duplicates selection

- (c) 2019, 2020 Dr. Ramil Nugmanov;
- (c) 2019 Dr. Timur Madzhidov; Ravil Mukhametgaleev

Installation instructions of CGRtools package information and tutorial's files see on `https://github.com/cimm-kzn/CGRtools`

NOTE: Tutorial should be performed sequentially from the start. Random cell running will lead to unexpected results. 

In [1]:
import pkg_resources
if pkg_resources.get_distribution('CGRtools').version.split('.')[:2] != ['4', '0']:
    print('WARNING. Tutorial was tested on 4.0 version of CGRtools')
else:
    print('Welcome!')

Welcome!


In [2]:
# load data for tutorial
from pickle import load
from traceback import format_exc

with open('molecules.dat', 'rb') as f:
    molecules = load(f) # list of MoleculeContainer objects
with open('reactions.dat', 'rb') as f:
    reactions = load(f) # list of ReactionContainer objects

m1, m2, m3 = molecules[:3] # molecule
m7 = m3.copy()
m11 = m3.copy()
m11.standardize()
m7.standardize()
r1 = reactions[0] # reaction
m1.delete_atom(3) 
cgr2 = ~r1
benzene = m3.substructure([4,5,6,7,8,9]) 
m3.delete_bond(4, 5)

## 2.1. Molecule Signatures
*MoleculeContainer* has methods for unique molecule signature generation.
Signature is SMILES string with canonical atoms ordering.

For signature generation one need to call `str` function on MoleculeContainer object.  
Fixed length hash of signature could be retrieved by calling `bytes` function on molecule (correspond to SHA 512 bitstring).

Order of atoms calculated by Morgan-like algorithm. On initial state for each atoms it's integer code calculated based on its type. All bonds incident to atoms also coded as integers and stored in sorted tuple. Atom code and tuple of it's bonds used for ordering and similar atoms detecting. Ordered atoms rank is replaced with new integer code. Atoms of the same type with the same bonds types incident to it have equal numbers.

Numbers codes found are used in Morgan algorithm cycle.
Loop is repeated until all atoms will be unique or number of unique atoms will not change in 3 subsequent loops.

In [3]:
ms2 = str(m2)  # get and print signature
print(ms2)  
# or 
print(m2)

hms2 = bytes(m2)  # get sha512 hash of signature as bytes-string
m2

C(C(=O)[O-])(=O)O.[Na+]
C(C(=O)[O-])(=O)O.[Na+]


String formatting is supported that is useful for reporting

In [4]:
print(f'f string {m2}')  # use signature in string formatting
print('C-style string %s' % m2)
print('format method {}'.format(m2))

f string C(C(=O)[O-])(=O)O.[Na+]
C-style string C(C(=O)[O-])(=O)O.[Na+]
format method C(C(=O)[O-])(=O)O.[Na+]


For Queries number of neighbors and hybridization will be added to signature. Note that in this case they are not readable as SMILES.
But possible to hide this data.

In [5]:
mq = m2.substructure(m2, as_query=True)
print(f'{mq}')  # get signatures with hybridization and neighbors data
print('{:!n}'.format(mq))  # get signature with hybridization only data
# h - hybridization marks, n- neighbors marks
print(format(mq, '!h')) # include only number of neighbors in signature
print(f'{mq:!n!h}')  # hide all data
mq

[C;d3;]([C;d3;]([O;s1;-])=[O;d1;])(=[O;d1;])[O;s1;].[Na;s0;+]
[C;d;]([C;d;]([O;s;-])=[O;d;])(=[O;d;])[O;s;].[Na;s;+]
[C;3;]([C;3;]([O;1;-])=[O;1;])(=[O;1;])[O;1;].[Na;0;+]
[C]([C]([O-])=[O])(=[O])[O].[Na+]


Atoms in the QueryContainer and QueryCGRContainer signature are represented in the following way: [isotope;element_symbol;hn;charge;radical state]. h mean hybridization, n - number of neighbors. Notation for hybridization is the following:

    s - all bonds of atom are single
    d - atom has one double bond and others are single
    t - atom has one triple or two double bonds and other are single
    a - atom is in aromatic ring

Examples: s1 - atom has s hybridization and one neighbor
d3 - atom has d hybridization and 3 neighbors

Signatures for CGRContainer include only radical state marks additionally to common SMILES notation.

Molecules comparable and hashable

Comparison of MoleculeContainer is based on its signatures. Moreover, since strings in Python are hashable, MoleculeContaier also hashable.

NOTE: MoleculeContainer can be changed. This can lead to unobvious behavior of the sets and dictionaries in which these molecules were placed before the change. Avoid changing molecules (standardize, aromatize, hydrogens and atoms/bonds changes) placed inside sets and dictionaries.

In [6]:
m1

In [7]:
m2

In [8]:
m1 != m2 # different molecules

True

In [9]:
m7

In [10]:
m7 == m11 # copy of the same molecule

True

In [11]:
m7 is m11  # this is not same objects!

False

In [12]:
# Simplest way to exclude duplicated structures
len({m1, m2, m7, m11}) == 3 # create set of unique molecules. Only 3 of them were different.

True

## 2.2. Reaction signatures
ReactionContainer have its signature. Signature is SMIRKS string in which molecules of reactants, reagents, products presented in canonical order.

API is the same as for molecules


In [13]:
str(r1)

'C(C(O)=O)(O)=O.C(O)C.C([O-])C>>C(C(OCC)=O)(OCC)=O'

## 2.3. CGR signature
CGRContainer have its signature. Signatures is SMIRKS-like strings where dynamic bond labels and dynamic atoms are also specified within squared brackets, so not only atoms but bonds could be written in brackets if a bond has complex parameters. Dynamic bonds in CGR have special label representing changes in bond orders. Dynamic atom corresponds to a change of formal charge or radical state of atom in reaction. Their labels are also given in brackets, including the atom symbol and text keys for atomic property in reactant and product, separated by symbol >. For a neutral atom A gaining a positive charge +n in reaction dynamic atom will be encoded as [A0>+n]. In case of charges +1 and -1, the number 1 is omitted. Properties for charges and radicals may be combined consecutively within one pair of brackets, e.g. [A0>-^>*] stands for an atom which becomes an anion-radical.

In [14]:
str(cgr2)

'C(C([.>-]OCC)([->.]O)=O)([.>-][O->0]CC)(=O)[->.]O'

In [15]:
cgr2.clean2d()
cgr2